Imports we'll need:

In [1]:
import torch
import torch.nn as nn

We can start by creating some sample data using `torch.tensor` (this works the same as `np.array`). 

Let's consider the data where the independent variable represents the number of hours studied and how much time spent sleeping, and the dependent variable represents the grade. 

In [2]:
x = torch.tensor(([2,9], \
                  [1,5], \
                  [3,6]),dtype=torch.float)
y = torch.tensor(([92], \
                 [100],\
                 [89]),dtype=torch.float)

x_test = torch.tensor(([4,8]),dtype=torch.float)

Check the size of the tensors we just created using the `size()` method.

In [3]:
print(x.size())
print(y.size())

torch.Size([3, 2])
torch.Size([3, 1])


We perform some scaling on the sample data.

In [5]:
x_max, _ = torch.max(x, 0)
x_test_max, _ = torch.max(x_test,0)

x = torch.div(x, x_max)
x_test = torch.div(x_test, x_test_max)
y = y / 100

Now we define the neural network class, which must inherit from `nn.Module`

In [6]:
class Neural_Network(nn.Module):
    def __init__(self, inputSize=2, outputSize=1, hiddenSize=3):
        super(Neural_Network, self).__init__()
        self._inputSize = inputSize
        self._outputSize = outputSize
        self._hiddenSize = hiddenSize
        
        self.w1 = torch.randn(self._inputSize, self._hiddenSize) # 3x2
        self.w2 = torch.randn(self._hiddenSize, self._outputSize) # 3x1
        
    def forward(self, x):
        self.z = torch.matmul(x, self.w1) # 3x3 dot does not broadcast in PyTorch
        self.z2 = self.sigmoid(self.z) # sigmoid activation
        self.z3 = torch.matmul(self.z2, self.w2)
        o = self.sigmoid(self.z3)
        return o
    
    def sigmoid(self,s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        """
        the derivative of the sigmoid
        """
        return s * (1-s)
    
    def backward(self, X, y, o):
        self.o_error = y - o
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.w2))
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        self.w1 += torch.matmul(torch.t(X), self.z2_delta)
        self.w2 += torch.matmul(torch.t(self.z2), self.o_delta)
        
    def train(self, X, y):
        o = self.forward(X)
        self.backward(X,y,o)
        
    def saveWeights(self, model):
        torch.save(model, "NN")
        # later you can load model with all the weights using:
        # torch.load("NN")
        
    def predict(self):
        print("Predicted data based on trained weights:")
        print("Input (scaled): {}".format(x_test))
        print("Output: {}".format(self.forward(x_test)))

Then a training loop could look like:

In [7]:
NN = Neural_Network()

def mean_square_loss(model, X, y):
    return torch.mean((y - model(X))**2).detach().item()


for i in range(1000):
    print("#{} Loss: {} ".format(i,mean_square_loss(NN, x, y)))
    
NN.saveWeights(NN)
NN.predict()

#0 Loss: 0.17397649586200714 
#1 Loss: 0.17397649586200714 
#2 Loss: 0.17397649586200714 
#3 Loss: 0.17397649586200714 
#4 Loss: 0.17397649586200714 
#5 Loss: 0.17397649586200714 
#6 Loss: 0.17397649586200714 
#7 Loss: 0.17397649586200714 
#8 Loss: 0.17397649586200714 
#9 Loss: 0.17397649586200714 
#10 Loss: 0.17397649586200714 
#11 Loss: 0.17397649586200714 
#12 Loss: 0.17397649586200714 
#13 Loss: 0.17397649586200714 
#14 Loss: 0.17397649586200714 
#15 Loss: 0.17397649586200714 
#16 Loss: 0.17397649586200714 
#17 Loss: 0.17397649586200714 
#18 Loss: 0.17397649586200714 
#19 Loss: 0.17397649586200714 
#20 Loss: 0.17397649586200714 
#21 Loss: 0.17397649586200714 
#22 Loss: 0.17397649586200714 
#23 Loss: 0.17397649586200714 
#24 Loss: 0.17397649586200714 
#25 Loss: 0.17397649586200714 
#26 Loss: 0.17397649586200714 
#27 Loss: 0.17397649586200714 
#28 Loss: 0.17397649586200714 
#29 Loss: 0.17397649586200714 
#30 Loss: 0.17397649586200714 
#31 Loss: 0.17397649586200714 
#32 Loss: 0.173976

/home/joseph/miniconda3/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Neural_Network. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
